In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import pickle
import numpy as np
from nltk import word_tokenize, FreqDist
import string
from os import path
from PIL import Image
import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS,ImageColorGenerator
# from nltk.sentiment.sentiment_analyzer import SentimentAnalyzer as sa
# from nltk.classify.scikitlearn import SklearnClassifier
# from nltk.classify import ClassifierI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise.model_selection import train_test_split, cross_validate
from surprise import Dataset, Reader, accuracy, SVD, NMF
reader= Reader(rating_scale=(1, 5.0))
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import Trainset
import requests

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

with open('userratingsDF.pickle', 'rb') as f:
    dfr = pickle.load(f)
with open('CategoricalItemDF.pickle', 'rb') as f:
    full = pickle.load(f)
with open('_SENT&RATINGDF.pickle', 'rb') as f:
    df1 = pickle.load(f)
with open('recdf.pickle', 'rb') as f:
    rec = pickle.load(f)
with open('ratingsmeandf.pickle', 'rb') as f:
    ratemean = pickle.load(f)

# Beginning Recommender System Using Just UserId, ItemID, and Rating

#### Item Based Collaborative Filtering probably better than User Based bc we have more users than items

In [2]:
toadd=full[['url','prodName','age','darkcircles','dry','redness','sensitive', 'oily', 'normal', 'cleanser',
       'exfoliator', 'makeup-removers', 'toner', 'mist', 'treatment', 'serum',
       'lotion', 'moisturizer', 'balm', 'oil', 'mask', 'peel', 'lip', 'eye',
       'supplement', 'tool']]

In [3]:
rec=rec.merge(toadd,on='url').drop_duplicates()
rec=rec.merge(ratemean,on='url').drop_duplicates()
rec.rename(columns={'rating_x':'rating','prodName_x':'brandName','prodName_y':'prodName'},inplace=True)
rec.drop(columns='rating_y',inplace=True)
rec.head()

,brandName,rating,url,user,prodName,age,darkcircles,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,rating_count
0,Paula's Choice,5,11174178,becky c,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
1,Paula's Choice,5,11174178,jess,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
2,Paula's Choice,5,11174178,lisa,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
3,Paula's Choice,5,11174178,stella2,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
4,Paula's Choice,5,11174178,carli,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60


In [4]:
data = Dataset.load_from_df(rec[['user','url','rating']],reader)

In [5]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  1378 

Number of items:  514


## Trying to Minimize RMSE

#### SVD Grid Search

In [8]:
params = {'n_factors' :[10,20,30,40,50,60,70,80,90,100,200],
         'reg_all':[0.02,0.05,0.1,0.4,0.5,0.6],'n_epochs': [5, 10],
         'lr_all': [0.002, 0.005],'reg_all': [0.4, 0.6]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [9]:
print(g_s_svd.best_score['rmse'])
print(g_s_svd.best_params['rmse'])

0.9638819259999204
{'n_factors': 70, 'reg_all': 0.6, 'n_epochs': 10, 'lr_all': 0.005}


#### NMF Grid Search

In [10]:
params = {'n_factors' :[5,10,20,30,40,50,60,70,80,90,100,200],
          'n_epochs': [5, 10]}
g_s_nmf = GridSearchCV(NMF,param_grid=params,n_jobs=-1)
g_s_nmf.fit(data)

In [11]:
print(g_s_nmf.best_score['rmse'])
print(g_s_nmf.best_params['rmse'])

0.9911537843911276
{'n_factors': 50, 'n_epochs': 10}


## KNN

### SIM_OPTIONS: Cosine Item Based

#### KNN Basic

In [12]:
knn_basic = KNNBasic(sim_options={'name':'cosine','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.96510296, 1.02006719, 1.00087647, 0.98523497, 0.91432779]))
('test_mae', array([0.67085589, 0.69634085, 0.6891606 , 0.68001497, 0.64467846]))
('fit_time', (0.00549006462097168, 0.004903316497802734, 0.004493236541748047, 0.00545191764831543, 0.005447864532470703))
('test_time', (0.0021009445190429688, 0.0016088485717773438, 0.0016167163848876953, 0.0063631534576416016, 0.0016210079193115234))
-----------------------
0.9771218741310397


#### KNN Baseline

In [13]:
knn_baseline = KNNBaseline(sim_options={'name':'cosine','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.15170462, 0.89021567, 0.84779518, 0.95569293, 0.96127369]))
('test_mae', array([0.75393756, 0.63315545, 0.63086167, 0.66858019, 0.66416234]))
('fit_time', (0.009792089462280273, 0.010627985000610352, 0.010663032531738281, 0.012160062789916992, 0.00857400894165039))
('test_time', (0.002145051956176758, 0.0018911361694335938, 0.0019118785858154297, 0.0019600391387939453, 0.0019221305847167969))
-----------------------
0.9613364186518266


#### KNN With Means

In [14]:
knn_means = KNNWithMeans(sim_options={'name':'cosine','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.07814294, 1.08698921, 0.90607688, 1.0301826 , 0.9158995 ]))
('test_mae', array([0.70790128, 0.71429661, 0.62723608, 0.69838652, 0.60874764]))
('fit_time', (0.011319875717163086, 0.0093231201171875, 0.010683298110961914, 0.008897066116333008, 0.0076580047607421875))
('test_time', (0.002398967742919922, 0.0019989013671875, 0.00202178955078125, 0.0018768310546875, 0.0018999576568603516))
-----------------------
1.0034582249283635


### SIM_OPTIONS: Cosine USer Based

#### KNN Basic

In [15]:
knn_basic = KNNBasic(sim_options={'name':'cosine','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.96733461, 1.00491385, 0.97217673, 0.97287796, 0.98763701]))
('test_mae', array([0.67635388, 0.69236676, 0.67643605, 0.66705735, 0.6764495 ]))
('fit_time', (0.037676095962524414, 0.036229848861694336, 0.03546404838562012, 0.04080510139465332, 0.03586387634277344))
('test_time', (0.0022611618041992188, 0.0021202564239501953, 0.0021009445190429688, 0.0022020339965820312, 0.002073049545288086))
-----------------------
0.980988033623302


#### KNN Baseline

In [16]:
knn_baseline = KNNBaseline(sim_options={'name':'cosine','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.93024808, 0.91860737, 0.92631218, 0.9538185 , 1.0953466 ]))
('test_mae', array([0.64804469, 0.64338432, 0.6536698 , 0.6698404 , 0.72201458]))
('fit_time', (0.0452730655670166, 0.044621944427490234, 0.029886960983276367, 0.03223776817321777, 0.03243064880371094))
('test_time', (0.0023369789123535156, 0.001973867416381836, 0.0018911361694335938, 0.0020110607147216797, 0.0020029544830322266))
-----------------------
0.9648665459936996


#### KNN With Means

In [17]:
knn_means = KNNWithMeans(sim_options={'name':'cosine','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.00454254, 0.98405371, 0.94390494, 1.11030994, 1.04431106]))
('test_mae', array([0.66548136, 0.65311712, 0.62759302, 0.7345523 , 0.67628336]))
('fit_time', (0.03988313674926758, 0.03989362716674805, 0.03481721878051758, 0.0306093692779541, 0.03368711471557617))
('test_time', (0.0022420883178710938, 0.0020711421966552734, 0.0020558834075927734, 0.0019178390502929688, 0.002053976058959961))
-----------------------
1.0174244362310738


### SIM_OPTIONS Peasrson Item Based

#### KNN Basic

In [18]:
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.95029582, 0.96349985, 0.9660831 , 1.10552879, 0.89976909]))
('test_mae', array([0.66361289, 0.67132203, 0.67414572, 0.73720822, 0.64676037]))
('fit_time', (0.008664846420288086, 0.009668827056884766, 0.004474163055419922, 0.004889965057373047, 0.004173994064331055))
('test_time', (0.0019290447235107422, 0.0019521713256835938, 0.001809835433959961, 0.0018329620361328125, 0.0016820430755615234))
-----------------------
0.9770353301207659


#### KNN Baseline

In [19]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.02107835, 0.97974722, 0.86214725, 0.9534784 , 0.98935933]))
('test_mae', array([0.69192237, 0.66502887, 0.62569067, 0.67400107, 0.68001619]))
('fit_time', (0.01643085479736328, 0.013280868530273438, 0.011008024215698242, 0.013643980026245117, 0.012448787689208984))
('test_time', (0.002260923385620117, 0.0018889904022216797, 0.001886129379272461, 0.001857757568359375, 0.001833200454711914))
-----------------------
0.9611621086944357


#### KNN With Means

In [20]:
knn_means = KNNWithMeans(sim_options={'name':'pearson','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.15183399, 0.86509733, 0.97637893, 0.96141385, 1.00813605]))
('test_mae', array([0.75774892, 0.61351048, 0.63181196, 0.65593807, 0.67402911]))
('fit_time', (0.010596036911010742, 0.014426708221435547, 0.013889074325561523, 0.010707855224609375, 0.014451026916503906))
('test_time', (0.002550840377807617, 0.0026369094848632812, 0.0025358200073242188, 0.0027289390563964844, 0.002588033676147461))
-----------------------
0.9925720318655313


### SIM_OPTIONS Pearson User Based

#### KNN Basic

In [21]:
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.94730587, 0.9827203 , 1.00606376, 1.00326014, 0.95218642]))
('test_mae', array([0.66090125, 0.68168454, 0.68977016, 0.68706748, 0.67121446]))
('fit_time', (0.04894685745239258, 0.051380157470703125, 0.04965710639953613, 0.029664039611816406, 0.046790122985839844))
('test_time', (0.0027170181274414062, 0.0021338462829589844, 0.0021741390228271484, 0.002131938934326172, 0.0020999908447265625))
-----------------------
0.9783072999111123


#### KNN Basline

In [22]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.87673456, 1.06330376, 0.9379797 , 1.00026955, 0.94443028]))
('test_mae', array([0.63118373, 0.72223799, 0.65817317, 0.67518544, 0.66052151]))
('fit_time', (0.0659189224243164, 0.03593611717224121, 0.04660987854003906, 0.04013824462890625, 0.04116511344909668))
('test_time', (0.0019903182983398438, 0.0019371509552001953, 0.0018641948699951172, 0.002012968063354492, 0.0020978450775146484))
-----------------------
0.9645435699730888


#### KNN With Means

In [23]:
knn_means = KNNWithMeans(sim_options={'name':'pearson','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.92660777, 1.03336675, 0.98567211, 1.00747201, 1.07442219]))
('test_mae', array([0.64499967, 0.67304538, 0.65975952, 0.64667387, 0.69990707]))
('fit_time', (0.03945302963256836, 0.04025721549987793, 0.038849592208862305, 0.06206512451171875, 0.04010200500488281))
('test_time', (0.002161741256713867, 0.002010822296142578, 0.002006053924560547, 0.0020308494567871094, 0.0020580291748046875))
-----------------------
1.0055081673471955


### SIM_OPTIONS MSD Item Based

#### KNN Basic

In [24]:
knn_basic = KNNBasic(sim_options={'name':'msd','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.01647978, 1.01043856, 0.95016539, 0.95155706, 0.96463292]))
('test_mae', array([0.68605454, 0.69391382, 0.66209497, 0.66528918, 0.67944351]))
('fit_time', (0.0036118030548095703, 0.002958059310913086, 0.004007101058959961, 0.0028951168060302734, 0.0038208961486816406))
('test_time', (0.0019910335540771484, 0.002017974853515625, 0.0016567707061767578, 0.002067089080810547, 0.0019872188568115234))
-----------------------
0.9786547437457042


#### KNN Baseline

In [25]:
knn_baseline = KNNBaseline(sim_options={'name':'msd','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.83996751, 0.9813342 , 1.00125397, 1.02373733, 0.98719241]))
('test_mae', array([0.60546693, 0.67422417, 0.69572056, 0.69322382, 0.67475406]))
('fit_time', (0.009591102600097656, 0.009470939636230469, 0.008480072021484375, 0.008943319320678711, 0.008273124694824219))
('test_time', (0.0023651123046875, 0.002210855484008789, 0.0018889904022216797, 0.0019118785858154297, 0.0018570423126220703))
-----------------------
0.9666970838593534


#### KNN With Means

In [26]:
knn_means = KNNWithMeans(sim_options={'name':'msd','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.06148837, 0.9805201 , 1.04135463, 0.99963221, 0.91451759]))
('test_mae', array([0.71790188, 0.65705579, 0.68392708, 0.66334085, 0.63103628]))
('fit_time', (0.009341955184936523, 0.009872913360595703, 0.008633852005004883, 0.009071111679077148, 0.008301019668579102))
('test_time', (0.0025670528411865234, 0.002282857894897461, 0.0020012855529785156, 0.002017974853515625, 0.0019321441650390625))
-----------------------
0.9995025817138016


### SIM_OPTIONS MSD USER BASED

#### KNN Basic

In [27]:
knn_basic = KNNBasic(sim_options={'name':'msd','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.95929827, 1.0341434 , 0.95203379, 1.0532034 , 0.90543573]))
('test_mae', array([0.67525055, 0.69685181, 0.65902327, 0.71572573, 0.64906639]))
('fit_time', (0.02712392807006836, 0.025660037994384766, 0.02218794822692871, 0.014955997467041016, 0.024418115615844727))
('test_time', (0.0021238327026367188, 0.002104043960571289, 0.0019259452819824219, 0.001895904541015625, 0.002624034881591797))
-----------------------
0.9808229194888082


##  ** KNN Baseline BEST RMSE

In [28]:
knn_baseline = KNNBaseline(sim_options={'name':'msd','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.77070108, 0.98647005, 1.01910444, 1.0088772 , 1.01443909]))
('test_mae', array([0.58677326, 0.677757  , 0.69136765, 0.69656059, 0.69191876]))
('fit_time', (0.039061784744262695, 0.029993772506713867, 0.022816181182861328, 0.03359365463256836, 0.022349119186401367))
('test_time', (0.002177000045776367, 0.002024412155151367, 0.0019457340240478516, 0.001986980438232422, 0.0019259452819824219))
-----------------------
0.9599183707126693


#### KNN With Means

In [29]:
knn_means = KNNWithMeans(sim_options={'name':'msd','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.01697338, 0.94979212, 1.11733684, 0.93285427, 1.14131015]))
('test_mae', array([0.69268544, 0.65378495, 0.71594177, 0.61841885, 0.7243519 ]))
('fit_time', (0.034214019775390625, 0.03357219696044922, 0.03748607635498047, 0.0278017520904541, 0.02735590934753418))
('test_time', (0.002546072006225586, 0.002099752426147461, 0.002073049545288086, 0.0019881725311279297, 0.0019598007202148438))
-----------------------
1.0316533538828536


### SIM_OPTIONS Pearson Baseline Item Based

#### KNN Basic

In [30]:
knn_basic = KNNBasic(sim_options={'name':'pearson_baseline','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.99762709, 0.94861526, 0.93941986, 1.00004146, 1.00860614]))
('test_mae', array([0.68341685, 0.67121446, 0.65663882, 0.69215461, 0.69114839]))
('fit_time', (0.009763002395629883, 0.008925914764404297, 0.0061016082763671875, 0.006119251251220703, 0.0060002803802490234))
('test_time', (0.0017008781433105469, 0.0015490055084228516, 0.0014529228210449219, 0.0015628337860107422, 0.0014209747314453125))
-----------------------
0.9788619631666917


#### ** KNN Baseline !!!! BEST RMSE B4

In [31]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson_baseline','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.9214284 , 1.06574318, 0.96935872, 1.0323361 , 0.84537739]))
('test_mae', array([0.63176772, 0.72333046, 0.68321741, 0.68290716, 0.63080957]))
('fit_time', (0.01288604736328125, 0.011952877044677734, 0.010934114456176758, 0.010596990585327148, 0.009831905364990234))
('test_time', (0.0025250911712646484, 0.0022079944610595703, 0.0018439292907714844, 0.0018270015716552734, 0.00180602073669433

#### KNN With Means

In [32]:
knn_means = KNNWithMeans(sim_options={'name':'pearson_baseline','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.95166268, 0.98237081, 0.9271081 , 0.98695442, 1.15669147]))
('test_mae', array([0.65317684, 0.66650664, 0.63698376, 0.67066357, 0.74737119]))
('fit_time', (0.014395952224731445, 0.014134883880615234, 0.014801263809204102, 0.012860774993896484, 0.01702117919921875))
('test_time', (0.0022699832916259766, 0.0018541812896728516, 0.0019168853759765625, 0.0015959739685058594, 0.00183558464050292

### SIM_OPTIONS Pearson Baseline User Based

#### KNN Basic

In [33]:
knn_basic = KNNBasic(sim_options={'name':'pearson_baseline','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.83709578, 1.00679858, 1.02471017, 0.97789789, 1.03711107]))
('test_mae', array([0.62201495, 0.67906702, 0.70773423, 0.68427516, 0.69982789]))
('fit_time', (0.03868603706359863, 0.03203105926513672, 0.027356863021850586, 0.03680300712585449, 0.03888583183288574))
('test_time', (0.0017778873443603516, 0.0017609596252441406, 0.0017099380493164062, 0.00177001953125, 0.001750946044921875))
-----------------------
0.9767226957927697


#### KNN Baseline

In [34]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson_baseline','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.86188808, 0.95059997, 1.05953237, 0.90892601, 1.02929891]))
('test_mae', array([0.62742803, 0.66851779, 0.70722505, 0.6437471 , 0.69638341]))
('fit_time', (0.031206130981445312, 0.027205944061279297, 0.026630878448486328, 0.029704809188842773, 0.026630163192749023))
('test_time', (0.0020599365234375, 0.0020170211791992188, 0.0019559860229492188, 0.0021021366119384766, 0.001995086669921875)

#### KNN With Means

In [35]:
knn_means = KNNWithMeans(sim_options={'name':'pearson_baseline','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.03491112, 0.88679591, 1.05585448, 1.09688131, 1.04779594]))
('test_mae', array([0.69445032, 0.59666266, 0.70458006, 0.70398574, 0.68272138]))
('fit_time', (0.04161977767944336, 0.03638291358947754, 0.03656792640686035, 0.03839707374572754, 0.03567790985107422))
('test_time', (0.0018727779388427734, 0.0017473697662353516, 0.0018420219421386719, 0.0017931461334228516, 0.0016598701477050781))

# Looking at predictions for best classification model

In [6]:
train, test = train_test_split(data,test_size=0.2)

In [7]:
# sim_options={'name':'pearson_baseline','user_based':False}
sim_options={'name':'msd','user_based':True}
knn_baseline = knns.KNNBaseline(sim_options=sim_options)
knn_baseline.fit(dataset)
predictions = knn_baseline.test(test)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.3164
0.31635538650751177


In [41]:
# knn_baseline.test(test)

# Collaborative Filtering, Item Based

In [162]:
rec.columns

Index(['brandName', 'rating', 'url', 'user', 'prodName', 'age', 'darkcircles',
       'dry', 'redness', 'sensitive', 'oily', 'normal', 'cleanser',
       'exfoliator', 'makeup-removers', 'toner', 'mist', 'treatment', 'serum',
       'lotion', 'moisturizer', 'balm', 'oil', 'mask', 'peel', 'lip', 'eye',
       'supplement', 'tool', 'rating_count'],
      dtype='object')

In [ ]:
# skins=['oily', 'dry', 'sensitive','normal']
# skincols=['oily','dry','sensitive','normal']
# typs=['cleanser','exfoliator', 'makeup-removers', 'toner', 'mist', 'treatment', 'serum','lotion', 'moisturizer', 
#       'balm', 'oil', 'mask', 'peel', 'lip', 'eye','supplement', 'tool']
# issues=['age', 'darkcircles','dry', 'dryness','redness', 'sensitive','oily']

## Trying to Predict for New Users

In [21]:
new_df=rec[['user','url','rating']]

In [66]:
def skin_rater(df,num, typ=None):
    rating_list = []
    movie_df=pd.DataFrame()
    raw_uid=input('type username: ')
    skintype=input('what is your skin type? (oily, dry, sensitive, normal, combination): ')
    issues=input('type any skin problems you have separated by commas (aging, dark circles, redness): ')
    prodtypes=input('is there any specific type of product you are missing? (cleanser, moisturizer, exfoliator, etc.): ')
    if skintype=='oily':
        movie_df.append(df[df['oily']==1])
    elif skintype=='sensitive':
        movie_df=df[df['sensitive']==1]
    elif skintype=='dry':
        movie_df=df[df['dry']==1]
    elif skintype=='normal' or skintype=='combination':
        movie_df=df[df['normal']==1]
    else:
        movie_df=rec
    if 'aging' in issues:
        movie_df.append(df[df['age']==1])
    if 'dark circles' in issues:
        movie_df.append(df[df['darkcircles']==1])
    if 'redness' in issues:
        movie_df.append(df[df['redness']==1])
    if 'cleanser' in prodtypes:
        movie_df.append(df[df['cleanser']==1])
    if 'exfoliator' in prodtypes:
        movie_df.append(df[df['exfoliator']==1])
    if 'makeup-removers' or 'remover' or 'makeup remover' in prodtypes:
        movie_df.append(df[df['makeup-removers']==1])
    if 'toner' in prodtypes:
        movie_df.append(df[df['toner']==1])
    if 'mist' in prodtypes:
        movie_df.append(df[df['mist']==1])
    if 'treatment' in prodtypes:
        movie_df.append(df[df['treatment']==1])
    if 'serum' in prodtypes:
        movie_df.append(df[df['serum']==1])
    if 'lotion' in prodtypes:
        movie_df.append(df[df['lotion']==1])
    if 'moisturizer' in prodtypes:
        movie_df.append(df[df['moisturizer']==1])
    if 'balm' in prodtypes:
        movie_df.append(df[df['balm']==1])
    if 'oil' in prodtypes:
        movie_df.append(df[df['oil']==1])
    if 'mask' in prodtypes:
        movie_df.append(df[df['mask']==1])
    if 'peel' in prodtypes:
        movie_df.append(df[df['peel']==1])
    if 'lip' in prodtypes:
        movie_df.append(df[df['lip']==1])
    if 'eye' in prodtypes:
        movie_df.append(df[df['eye']==1])
    if 'supplement' or 'pill' or 'pills' in prodtypes:
        movie_df.append(df[df['supplement']==1])
    if 'tool' in prodtypes:
        movie_df.append(df[df['tool']==1])
    else:
        movie_df=df
    movie_df.drop_duplicates(inplace=True)
    while num > 0:
        p = movie_df.sample(1)
        print(p[['brandName','prodName','url']])
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'user':raw_uid,'url':p['url'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return(rating_list,raw_uid)
def rank_prods_for_user(df,raw_uid):
    list_of_prods=[]
    for u in df['url'].unique():
            list_of_prods.append((u,knn_baseline.predict(raw_uid,u)[3]))
    ranked_prods = sorted(list_of_prods,key=lambda x:x[1],reverse=True)
    return (ranked_prods)
def recommended_products(user_ratings,df,n):
    for idx, re in enumerate(user_ratings):
        u=re[0]
        title = df.loc[df['url'] == int(re[0])]['prodName']
        print('Recommendation # ',idx+1,'|| ', 'url:', u, '||', 'product: ', df[df['url']==u]['prodName'].drop_duplicates(),'\n')
        n-= 1
        if n == 0:
            break

In [67]:
lis,uid=skin_rater(rec,3)

type username: noooo
what is your skin type? (oily, dry, sensitive, normal, combination): no
type any skin problems you have separated by commas (aging, dark circles, redness): no
is there any specific type of product you are missing? (cleanser, moisturizer, exfoliator, etc.): no
      brandName                             prodName       url
494  SkinMedica  SkinMedica Vitamin C Plus E Complex  11289692
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1
     brandName                       prodName       url
1083    ARCONA  ARCONA Hydrating Serum 1.17oz  11286244
How do you rate this movie on a scale of 1-5, press n if you have not seen :
2
        brandName                                           prodName       url
277  The Ordinary  The Ordinary Hyaluronic Acid 2% + B5 Hydration...  11363395
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3


In [68]:
new_ratings_df=new_df.append(lis,ignore_index=True)
new_data= Dataset.load_from_df(new_df,reader)
new_data=new_data.build_full_trainset()
knn_baseline=knns.KNNBaseline(sim_options={'name':'pearson_baseline','user_based':False})
knn_baseline.fit(new_data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [69]:
ranked=rank_prods_for_user(new_df,uid)

In [70]:
recommended_products(ranked,rec,5)

Recommendation #  1 ||  url: 11286360 || product:  1061    Avène Thermal Spring Water 10.58oz
Name: prodName, dtype: object 

Recommendation #  2 ||  url: 10375555 || product:  174    Murad Pure Skin Clarifying Dietary Supplement ...
Name: prodName, dtype: object 

Recommendation #  3 ||  url: 11524845 || product:  1140    FOREO LUNA™ mini 2 - Fuchsia
Name: prodName, dtype: object 

Recommendation #  4 ||  url: 11524843 || product:  1485    FOREO LUNA™ 2 for Oily Skin
Name: prodName, dtype: object 

Recommendation #  5 ||  url: 10543756 || product:  1561    Dr. Brandt Pores No More Pore Refiner (1oz)
Name: prodName, dtype: object 



## To Personalize for skin problems or type of product...

### Put into function later!!

In [54]:
ranked=rank_prods_for_user(rec[rec['age']==1][['user','url','rating']],'mack')

In [55]:
recommended_products(ranked,rec,3)

Recommendation #  1 ||  url: 11429303 || product:  293    The Ordinary Glycolic Acid 7% Toning Solution ...
Name: prodName, dtype: object 

Recommendation #  2 ||  url: 11286857 || product:  598    Cellex-C Advanced C Serum
Name: prodName, dtype: object 

Recommendation #  3 ||  url: 11286872 || product:  612    Cellex-C Hydra 5 B-Complex
Name: prodName, dtype: object 



In [57]:
rec[rec['url']=='11429303']

,brandName,rating,url,user,prodName,age,darkcircles,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,rating_count
293,The Ordinary,3,11429303,nano,The Ordinary Glycolic Acid 7% Toning Solution ...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,14
294,The Ordinary,5,11429303,rachel,The Ordinary Glycolic Acid 7% Toning Solution ...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,14
295,The Ordinary,5,11429303,n.amal,The Ordinary Glycolic Acid 7% Toning Solution ...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,14
296,The Ordinary,5,11429303,mary,The Ordinary Glycolic Acid 7% Toning Solution ...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,14
297,The Ordinary,5,11429303,mack,The Ordinary Glycolic Acid 7% Toning Solution ...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,14
298,The Ordinary,5,11429303,ana carolina,The Ordinary Glycolic Acid 7% Toning Solution ...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,14
299,The Ordinary,5,11429303,ro,The Ordinary Glycolic Acid 7% Toning Solution ...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,14


## Description TFIDF and Cosine Similarity !?!??!?!?!??! LOOKING AT REVIEW CONTENT AND TITLES

In [57]:
df1['description']=df1['title']+df1['content']

In [58]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df1['description'])

In [59]:
tfidf_matrix.shape

(1685, 39382)

In [60]:
full.shape

(1701, 30)

In [61]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [62]:
cosine_sim

array([[1.        , 0.03513622, 0.02979474, ..., 0.00730602, 0.        ,
        0.00399307],
       [0.03513622, 1.        , 0.02854128, ..., 0.        , 0.        ,
        0.        ],
       [0.02979474, 0.02854128, 1.        , ..., 0.        , 0.        ,
        0.00425702],
       ...,
       [0.00730602, 0.        , 0.        , ..., 1.        , 0.        ,
        0.05258715],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.00154483],
       [0.00399307, 0.        , 0.00425702, ..., 0.05258715, 0.00154483,
        1.        ]])

In [63]:
# def item(id):  
#     return df1.loc[df1['url'] == id]['description'].tolist()
# # Just reads the results out of the dictionary.def 
# def recommend(item_id, num):
#     print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
#     print("-------")    
#     recs = results[item_id][:num]   
#     for rec in recs: 
#         print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [64]:
# df1.reset_index(inplace=True)
# products = df1['url']
# indices = pd.Series(df1.index, index=df1['url'])

In [65]:
# def get_recommendations(product):
#     idx = indices[products]
#     sim_scores = list(enumerate(cosine_sim[idx]))
# #     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:31]
#     prod_indices = [i[0] for i in sim_scores]
#     return products.iloc[prod_indices]

In [67]:
# get_recommendations(11429304).drop_duplicates()

In [68]:
# recommend(10362879,5)

In [69]:
alldf=dfr[['url','prodName','rating','user']]

In [70]:
alldf.groupby(['url','user','prodName']).mean().head()

rating
url      user      prodName        
10236357 adrian    Anthony        5
         george    Anthony        5
         kimber    Anthony        5
         mike      Anthony        5
         westtexas Anthony        5

In [44]:
train, test = train_test_split(data,test_size=0.2)

## Does this produce recommendations or is it messed up?

In [169]:
# def skincare_rater(df,num, typ=None, problem=None):
#     userID = 1000
#     rating_list = []
#     while num > 0:
#         if typ:
#             prod = df[df[''].str.contains(typ)].sample(1)
#         else:
#             sample=df.sample(1)
#             brand=sample[['brandName']]
#             product=sample[['prodName']]
#             url=sample[['url']]
#             prod = sample[['brandName','prodName','url']]
#         print(prod)
#         rating = input('How do you rate this product on a scale of 1-5, press n if you have not used :\n')
#         if rating == 'n':
#             continue
#         else:
#             rating_one_prod=df.sample(1)[['url','prodName']]
#             rating_one_prod['user']=1000
#             rating_one_prod['rating']=rating
#             ratedict={'url':url,'brandName':brand,'prodName':product}
#             rating_list.append(rating_one_prod) 
#             num -= 1
#     return rating_list